In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
'''
INTRODUCTION: 
In the domain of sports analytics, predicting the outcomes of football matches is a challenging yet valuable task. It enables clubs to anticipate game results, assists bettors in decision-making, and enriches fan experiences. This report details the development of  machine learning model designed to predict the results of Premier League football matches offering insights into team performance and match dynamics 

METHODOLOGY:
Our dataset comprises match statistics from the Premier League spanning from 2000 to 2022 seasons. We performed some data cleaning, handled missing values and encoded categorical variables. We engineered features like team point differences and goal statistics. We Evaluated logistic regression, Random Forest, Gradient Boosting models, optimizing Hyperparameter using Grid Search


RESULTS: 
The Gradient Boosting Model achieve the highest accuracy of 69.00% on the testing set. with a precision of  Precision: 67.49 %,  Recall: 81.47% and an F1 Score of 73.82 % . Cross validation yielded a mean accuracy of 63.44% with a standard deviation of 1.04 suggesting stable performance across folds. 


DISCUSSION: 
The Gradient Boosting model demonstrated moderate predictive accuracy, with room for improvement. The learning curve indicated that additional training data could further refine the model. Feature importance analysis highlighted 'DiffPts' as the most significant predictor, suggesting that relative team strength is a critical factor in match outcomes. Limitations include the model's sensitivity to feature selection and potential overfitting, as indicated by the disparity between training and validation performance


CONCLUSION: 
Our analysis confirms the potential of machine learning in predicting football match outcomes. While the current model provides a solid foundation, future work could explore more complex features, alternative modeling techniques like deep learning, and real-time data incorporation to enhance prediction accuracy. Continuing to refine the model could yield a valuable tool for strategists and enthusiasts alike
'''

In [ ]:
matches = pd.read_csv("matches.csv", index_col=0)

In [ ]:
matches.head()

In [ ]:
matches.shape

In [ ]:
# 2 seasons * 20 squads * 38 matches
2 * 20 * 38

In [ ]:
# Missing Liverpool 2021-2022
matches["team"].value_counts()
#3 teams get relegated every year 
#3 teams get promoted every year into premier league              

In [ ]:
# Realized there is a mission season for liverpool 
matches[matches["team"] == "Liverpool"].sort_values("date")

In [ ]:
matches["round"].value_counts()

In [ ]:
matches.dtypes

In [ ]:
# Cleaning column "comp"
del matches["comp"]

In [ ]:
#Cleaning column "notes"
del matches["notes"]

In [ ]:
# Converting date to data type in panda. 
matches["date"] = pd.to_datetime(matches["date"])

In [ ]:
# if result = true = winning if not lost or draw 
matches["target"] = (matches["result"] == "W").astype("int")

In [ ]:
matches

In [ ]:
# Predictor converting Home/ Away into a numeric column 0=Home 1=Away 
matches["venue_code"] = matches["venue"].astype("category").cat.codes

In [ ]:
# Unique code for each opponent team
matches["opp_code"] = matches["opponent"].astype("category").cat.codes

In [ ]:
# Maybe some time plays at better time of the day
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")

In [ ]:
# Day code for each day of the week 
matches["day_code"] = matches["date"].dt.dayofweek

In [ ]:
matches

In [ ]:
#Import randomforest classifier 
# picking up non linear data in the code. 
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [ ]:
# Making sure that all the test data come after the data of the training data
# taking all the matches before 2022
train = matches[matches["date"] < '2022-01-01']

In [ ]:
# Anything in 2022
test = matches[matches["date"] > '2022-01-01']

In [ ]:
# List of features columns we created.
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [ ]:
rf.fit(train[predictors], train["target"])

In [ ]:
preds = rf.predict(test[predictors])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
# Determine the accuracy of the model. 
accuracy = accuracy_score(test["target"], preds)

In [ ]:
accuracy

In [ ]:
# Combine our actual values vs predicted value 
combined = pd.DataFrame(dict(actual=test["target"], predicted=preds))

In [ ]:
# Create when we predicted a 0 and a 1 and what actually happened.
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

In [ ]:
'''
0 = predicted a loss or draw 
1 = Win 
'''

In [ ]:
from sklearn.metrics import precision_score
# WHen we predicted a win what percentage of time did the team actually win 
precision_score(test["target"], preds)

In [ ]:
# Create one dataframe for every team 
grouped_matches = matches.groupby("team")

In [ ]:
group = grouped_matches.get_group("Manchester City").sort_values("date")

In [ ]:
# Take a group in, it will take a set of columns to compute the rolling average and take a new set of new columns to assign rolling averages 
def rolling_averages(group, cols, new_cols):
    # Sort it in order of date
    group = group.sort_values("date")
    # take a set of columns and computing the rolling averages  closed="left"
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    # Dropping any missing values 
    group = group.dropna(subset=new_cols)
    return group

In [ ]:
# Columns we want to computer rolling average
# gf = goals for, ga= goal against, sh = shots taken, sot = shots on target, dist = distance shot travel, fk = freekick, pk = penalty kicks, pkatt= penalty kicks attends
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

rolling_averages(group, cols, new_cols)

In [ ]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

In [ ]:
matches_rolling

In [ ]:
matches_rolling = matches_rolling.droplevel('team')

In [ ]:
matches_rolling

In [ ]:
# assign value from 0 to 1316 to get unique values for each index. 
matches_rolling.index = range(matches_rolling.shape[0])

In [ ]:
# Applying changes for better accuracy score
def make_predictions(data, predictors):
    train = data[data["date"] < '2022-01-01']
    test = data[data["date"] > '2022-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    error = precision_score(test["target"], preds)
    return combined, error

In [ ]:
combined, error = make_predictions(matches_rolling, predictors + new_cols)

In [ ]:
# 47 to 62
error

In [ ]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)

In [ ]:
combined.head(10)

In [ ]:
class MissingDict(dict):
    # Create a class that inherit from the dict class 
    # panda will not handle any missing keys, it will just remove missing name. and replace it with the value below.  
    __missing__ = lambda self, key: key

map_values = {"Brighton and Hove Albion": "Brighton", 
              "Manchester United": "Manchester Utd", 
              "Newcastle United": "Newcastle Utd", 
              "Tottenham Hotspur": "Tottenham", 
              "West Ham United": "West Ham", 
              "Wolverhampton Wanderers": "Wolves"
              } 
mapping = MissingDict(**map_values)

In [ ]:
combined["new_team"] = combined["team"].map(mapping)

In [ ]:
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])

In [ ]:
merged

In [ ]:
# Importing a richer data set with 22 season 2000-2022
new_data_set = pd.read_csv("final_dataset.csv")

In [ ]:
print(f"Number of rows: {new_data_set.shape[0]}")
print(f"Number of cols: {new_data_set.shape[1]}")

In [ ]:
summary_statistics = new_data_set.describe()

In [ ]:
missing_values = new_data_set.isnull().sum()

In [ ]:
new_data_set.info()

In [ ]:
new_data_set.isnull().sum()

In [ ]:
# Check for duplicates 
new_data_set.duplicated().sum()

In [ ]:
# Dro rows with missing values 
new_data_set = new_data_set.dropna()

In [ ]:
# Setting up visualization parameters: 
sns.set(style="whitegrid")
# Prepare a figure for multiple plots: 
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(14, 18))
# Plotting distributions and relationships
sns.histplot(data=new_data_set, x='FTHG', bins=10, kde=True, ax=axes[0, 0])
axes[0, 0].set_title('Distribution of Full-time Home Goals')

sns.histplot(data=new_data_set, x='FTAG', bins=10, kde=True, ax=axes[0, 1])
axes[0, 1].set_title('Distribution of Full-time Away Goals')

sns.boxplot(x='FTR', y='HTP', data=new_data_set, ax=axes[1, 0])
axes[1, 0].set_title('Home Team Points vs Match Result')

sns.boxplot(x='FTR', y='ATP', data=new_data_set, ax=axes[1, 1])
axes[1, 1].set_title('Away Team Points vs Match Result')

sns.scatterplot(x='HTGD', y='ATGD', hue='FTR', data=new_data_set, ax=axes[2, 0])
axes[2, 0].set_title('Home vs. Away Team Goal Difference')


plt.delaxes(axes[2, 1])

plt.tight_layout()
plt.show()



In [ ]:
# Feature engineering: Creating interaction features 
new_data_set['H_vs_A_Goal_Scored_Diff'] = new_data_set['HTGS'] - new_data_set['ATGS']
new_data_set['H_vs_A_Goal_Conceded_Diff'] = new_data_set['HTGC'] - new_data_set['ATGC']
new_data_set['Points_Ratio'] = new_data_set['HTP'] / (new_data_set['ATP'] + 0.01)  # Adding a small constant to avoid division by zero

In [ ]:
# Initialize the LabelEncoder
label_encoder = LabelEncoder()
    # Encode the 'FTR' column where:
# 'A' (Away win) is encoded as 0, 'D' (Draw) as 1, 'H' (Home win) as 2
new_data_set['FTR_encoded'] = label_encoder.fit_transform(new_data_set['FTR'])
# Display the first few rows to verify the encoding
new_data_set[['FTR', 'FTR_encoded']].head()


In [ ]:
# Normalization: Scaling goal differences and total goals
scaler = MinMaxScaler()
new_data_set[['HTGD', 'ATGD', 'H_vs_A_Goal_Scored_Diff', 'H_vs_A_Goal_Conceded_Diff']] = scaler.fit_transform(
   new_data_set[['HTGD', 'ATGD', 'H_vs_A_Goal_Scored_Diff', 'H_vs_A_Goal_Conceded_Diff']])

In [ ]:
# Verifing the newly added Features 
new_data_set[['HTGD', 'ATGD', 'H_vs_A_Goal_Scored_Diff', 'H_vs_A_Goal_Conceded_Diff', 'Points_Ratio']].head()

In [ ]:
print(new_data_set.columns)

In [ ]:
# Preparing features and target variable
features = ['HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts',
            'HTFormPts', 'ATFormPts', 'H_vs_A_Goal_Scored_Diff', 'H_vs_A_Goal_Conceded_Diff', 'Points_Ratio']
X = new_data_set[features]
y = new_data_set['FTR_encoded']

In [ ]:
# Splitting the data into training and testing sets: 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize the models: 
logistic_model = LogisticRegression(max_iter=1000)
random_forest_model = RandomForestClassifier(n_estimators=100)
gradient_boosting_model = GradientBoostingClassifier(n_estimators=100)

In [ ]:
# Dictionary to Hold models and their performances 
model_performance = {}

def train_evaluate_model(model, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_train)
    accuracy = accuracy_score(y_train, y_pred)
    precision = precision_score(y_train, y_pred, average='weighted')
    recall = recall_score(y_train, y_pred, average='weighted')
    f1 = f1_score(y_train, y_pred, average='weighted')
    
    # Storing performance metrics
    model_performance[model_name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

# Train and evaluate each model
print("Training the model...")
train_evaluate_model(logistic_model, "Logistic Regression")
train_evaluate_model(random_forest_model, "Random Forest")
train_evaluate_model(gradient_boosting_model, "Gradient Boosting")

model_performance

In [ ]:
def test_evaluate_model(model, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Storing performance metrics
    model_performance[model_name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

# Train and evaluate each model
print("Testing model ")
test_evaluate_model(logistic_model, "Logistic Regression")
test_evaluate_model(random_forest_model, "Random Forest")
test_evaluate_model(gradient_boosting_model, "Gradient Boosting")

model_performance

In [ ]:
# Gradient Boosting was the model the performed better, so we went ahead and started the Hyperparameter tuning to improve the model performance 
from sklearn.model_selection import GridSearchCV

# Set up the parameter grid for Gradient Boosting
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 4],
    'min_samples_leaf': [1, 2]
}

# Initialize the GridSearchCV object
grid_search = GridSearchCV(estimator=GradientBoostingClassifier(random_state=42),
                           param_grid=param_grid,
                           scoring='accuracy',  # focusing on accuracy as the metric for simplicity
                           cv=3,  # using 3-fold cross-validation
                           verbose=1,  # printing out progress
                           n_jobs=-1)  # using all available CPUs

# Fit GridSearchCV to the training data
grid_search.fit(X_train, y_train)

# Best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

best_params, best_score


In [ ]:
# Re-training the Gradient Boosting model with the optimized parameters
optimized_gb_model = GradientBoostingClassifier(
    n_estimators=50,
    learning_rate=0.05,
    max_depth=3,
    min_samples_split=2,
    min_samples_leaf=2,
    random_state=42
)

# Fit the model to the training data
optimized_gb_model.fit(X_train, y_train)

# Predict on the training data
y_train_pred_optimized = optimized_gb_model.predict(X_train)

# Calculate the accuracy on the testing data
optimized_training_accuracy = accuracy_score(y_train, y_train_pred_optimized)

optimized_training_accuracy, 

In [ ]:
# Re-Testing the Gradient Boosting Model with the optimized parameters 
optimized_gb_model_test = GradientBoostingClassifier(
    n_estimators=50,
    learning_rate=0.05,
    max_depth=3,
    min_samples_split=2,
    min_samples_leaf=2,
    random_state=42
)
# Fit the model to the test data 
optimized_gb_model_test.fit(X_test, y_test)
# Predicting on the testing data 
y_test_pred_optimized = optimized_gb_model_test.predict(X_test)

# Calculating the accuracy: 
optimized_testing_accuracy = accuracy_score(y_test, y_test_pred_optimized)

# Calculate Precision
optimized_precision = precision_score(y_test, y_test_pred_optimized)

# Calculate Recall
optimized_recall = recall_score(y_test, y_test_pred_optimized)

# Calculate F1 Score
optimized_f1 = f1_score(y_test, y_test_pred_optimized)


print("Accuracy: ",optimized_testing_accuracy)

print("Precision:", optimized_precision)
print("Recall:", optimized_recall)
print("F1 Score:", optimized_f1)


In [ ]:
# Assuming 'optimized_gb_model' is your trained Gradient Boosting model and 'X' is your feature matrix
feature_importances = optimized_gb_model.feature_importances_

# Creating a DataFrame to hold the feature names and their corresponding importance
features_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

# Sorting the DataFrame by the 'Importance' column in descending order
features_df = features_df.sort_values(by='Importance', ascending=False)

# Plotting the feature importances
plt.figure(figsize=(10, 8))
sns.barplot(x='Importance', y='Feature', data=features_df)
plt.title('Feature Importance in Optimized Gradient Boosting Model')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

# Print the DataFrame to view it in tabular form
print(features_df)


In [ ]:
'''    DiffPts: Likely represents the difference in points between the two teams. Points could be the total accumulated over the season or a set of games.
    H_vs_A_Goal_Scored_Diff: Home vs Away Goal Scored Diff
    ATP:  "Away Team Points," 
    HTGD: "Home Team Goal Difference" 
    ATGD: "Away Team Goal Difference" 
    Points_Ratio: Likely the ratio of points between the home and away teams, providing a relative measure of their strengths.
    HTP: "Home Team Points" 
    H_vs_A_Goal_Conceded_Diff: 
    DiffFormPts: Likely represents the difference in form points between the two teams, which could be based on recent performances.
    ATFormPts: "Away Team Form Points"
    ATGC: "Away Team Goals Conceded" 
    HTGS: "Home Team Goals Scored"
    HTGC: "Home Team Goals Conceded" 
    HTFormPts: "Home Team Form Points"
    ATGS: "Away Team Goals Scored" 
    '''

In [ ]:
# Validation: 
# Using 5-fold cross-validation to validate the optimized Gradient Boosting model
cv_scores = cross_val_score(optimized_gb_model, X, y, cv=5, scoring='accuracy')

# Calculate the mean and standard deviation of the cross-validation scores
cv_mean = cv_scores.mean()
cv_std = cv_scores.std()

# Gives you an average score across all fold providing an insight into how generally well the model performs
print("Cross-Validation Mean Accuracy: {:.2f}%".format(cv_mean * 100))
# Indicates the varaibility of the model's performance, giving you an idea about its stabiliy across different subsets 
print("Cross-Validation Standard Deviation: {:.2f}%".format(cv_std * 100))

In [ ]:
from sklearn.model_selection import learning_curve

# Define function to plot learning curves
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1, color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

# Generate the learning curve for the optimized Gradient Boosting model
cv = 5  # 5-fold cross-validation
plot_learning_curve(optimized_gb_model, 'Learning Curve for Gradient Boosting', X, y, cv=cv, n_jobs=-1)


In [ ]:
def plot_roc_curve(model, X_test, y_test):
    # Predict probabilities for the positive class
    probs = model.predict_proba(X_test)[:, 1]
    # Calculate ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, probs)
    # Calculate AUC score
    auc = roc_auc_score(y_test, probs)
    
    # Plot
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.show()

# Example of plotting ROC curve for the Gradient Boosting model
plot_roc_curve(optimized_gb_model, X_test, y_test)

In [ ]:
def plot_confusion_matrix(model, X_test, y_test):
    # Predictions
    y_pred = model.predict(X_test)
    # Generate confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    # Plot
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

# Example of plotting Confusion Matrix for the Gradient Boosting model
plot_confusion_matrix(optimized_gb_model, X_test, y_test)

In [ ]:
'''
Label 0 represent Home loss or Not Win 
Label 1 represent Home Win 

Top Left True Negative 328: Represent correct precitions for the negative class. 
Top-Right False Positive 306: This represents the error where the model incorecctly predicted the positve class. 
Bottom-Left False Negative 180: This represents the error where the model incorrectly predicted the negative class 
Bottom Right: True positive 554: This represent the correct predictions for the positive class.
'''

In [ ]:
# Preparing features and encoding the target 'FTR'
from sklearn.preprocessing import LabelEncoder

# Encoding the target
label_encoder = LabelEncoder()
new_data_set['FTR_encoded'] = label_encoder.fit_transform(new_data_set['FTR'])

# Selecting a subset of features for simplicity
features = ['HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts']
X = new_data_set[features]
y = new_data_set['FTR_encoded']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Re-initialize and train the Random Forest and Gradient Boosting models
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)
gradient_boosting_model = GradientBoostingClassifier(n_estimators=50, learning_rate=0.05, max_depth=3, 
                                                     min_samples_split=2, min_samples_leaf=2, random_state=42)

# Train the models
random_forest_model.fit(X_train, y_train)
gradient_boosting_model.fit(X_train, y_train)

# Making predictions on the test set
y_pred_rf = random_forest_model.predict(X_test)
y_pred_gb = gradient_boosting_model.predict(X_test)

# Mapping predictions back to readable format
y_pred_rf_mapped = label_encoder.inverse_transform(y_pred_rf)
y_pred_gb_mapped = label_encoder.inverse_transform(y_pred_gb)

# Extracting actual team names and results from the test set
test_team_info = new_data_set.loc[X_test.index, ['HomeTeam', 'AwayTeam', 'FTR']]

# Creating the final DataFrame
final_predictions_with_actual_names = pd.DataFrame({
    'HomeTeam': test_team_info['HomeTeam'],
    'AwayTeam': test_team_info['AwayTeam'],
    'Actual Result': test_team_info['FTR'],
    'RF Prediction': y_pred_rf_mapped,
    'GB Prediction': y_pred_gb_mapped
})

final_predictions_with_actual_names


In [ ]:
'''
H = Home win
NH = Not Home Win 
'''